<a href="https://colab.research.google.com/github/schwarer2006/ASP.NET-Simple-Login-Application/blob/master/Komplexes_JOB_CV_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('punkt')  # Herunterladen des Tokenizers
nltk.download('stopwords')  # Herunterladen der Stopwörter

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
!pip install openai

In [32]:
import os
import re
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from matplotlib.backends.backend_pdf import PdfPages
from datetime import datetime

# NLTK-Ressourcen herunterladen
nltk.download('stopwords')
nltk.download('punkt')

# Laden und Bereinigen von Textdateien
def load_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read().lower()  # Direkt in Kleinbuchstaben konvertieren
    text = re.sub(r'\s+', ' ', text)  # Überflüssige Leerzeichen entfernen
    text = re.sub(r'[^\w\s]', '', text)  # Sonderzeichen entfernen
    return text

resume_text = load_text('/content/lebenslauf.txt')
job_description_text = load_text('/content/stellenbeschreibung.txt')

# Tokenisierung und Entfernung von Stoppwörtern
stop_words = set(stopwords.words('german'))
def tokenize_and_remove_stopwords(text):
    tokens = word_tokenize(text)
    return [word for word in tokens if word not in stop_words]

resume_tokens = tokenize_and_remove_stopwords(resume_text)
job_description_tokens = tokenize_and_remove_stopwords(job_description_text)

# Cosinus-Ähnlichkeit berechnen
def calculate_cosine_similarity(text1, text2):
    vectorizer = CountVectorizer()
    vectors = vectorizer.fit_transform([text1, text2]).toarray()
    return cosine_similarity(vectors)[0][1]

similarity_score = calculate_cosine_similarity(resume_text, job_description_text)

# Funktion zur Erstellung und Speicherung von Visualisierungen als PDF
def save_visualizations_to_pdf(similarity_score, name, vorname, stellenname):
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    pdf_path = f'/content/visualizations_{timestamp}.pdf'
    with PdfPages(pdf_path) as pdf:
        # Persönliche Informationen und Übereinstimmungsdaten auf einer Seite
        fig, ax = plt.subplots(figsize=(8, 6))
        info_text = (f"Name: {name}\nVorname: {vorname}\nStellenname: {stellenname}\n"
                     f"Datum: {datetime.now().strftime('%d.%m.%Y')}\n"
                     f"Übereinstimmung in Prozent: {similarity_score * 100:.2f}%")
        ax.text(0.5, 0.5, info_text, fontsize=12, ha='center', va='center', transform=ax.transAxes)
        ax.axis('off')
        pdf.savefig(fig)
        plt.close(fig)

        # Visualisierungen für Lebenslauf und Stellenbeschreibung
        for text, title in [(resume_text, 'Resume Wordcloud'), (job_description_text, 'Job Description Wordcloud')]:
            fig = plt.figure(figsize=(10, 5))
            wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
            plt.imshow(wordcloud, interpolation='bilinear')
            plt.axis('off')
            plt.title(title)
            pdf.savefig(fig)
            plt.close(fig)

        # Wortfrequenzdiagramme
        for tokens, title in [(resume_tokens, 'Resume Word Frequency'), (job_description_tokens, 'Job Description Word Frequency')]:
            fig = plt.figure(figsize=(10, 6))
            word_freq = Counter(tokens)
            df = pd.DataFrame(list(word_freq.items()), columns=['Word', 'Frequency']).sort_values('Frequency', ascending=False)
            sns.barplot(x='Frequency', y='Word', data=df.head(20), palette='viridis')
            plt.title(title)
            pdf.savefig(fig)
            plt.close(fig)

    print(f"PDF gespeichert unter: {pdf_path}")

# Beispieldaten
name = "Schwarz"
vorname = "Erik"
stellenname = "SQL Business Intelligence Entwickler"

# Visualisierungen in PDF speichern
save_visualizations_to_pdf(similarity_score, name, vorname, stellenname)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
<ipython-input-32-4c4ac825ae0c>:78: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='Frequency', y='Word', data=df.head(20), palette='viridis')
<ipython-input-32-4c4ac825ae0c>:78: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='Frequency', y='Word', data=df.head(20), palette='viridis')


PDF gespeichert unter: /content/visualizations_2024-04-19_08-30-27.pdf
